In [ ]:
import pandas as pd
import random
import time

import requests
from tqdm import tqdm     
from bs4 import BeautifulSoup
import mariadb

from selenium import webdriver

In [ ]:
def get_review(link):
    driver.get(link)
    driver.implicitly_wait(8) 
    time.sleep(2+random.random())

    page_source = driver.page_source
    html = BeautifulSoup(page_source, 'html.parser')

    # 댓글
    review=html.find_all('div',class_='cmt_text d_cmtpgn_cmt_full_contents')
    review_list=[]
    for i in review:
        try:
            review=(i.text.replace('내용',"").strip())
            review_list.append(review)
        except Exception as e:
            print(e)
            continue
    return review_list

In [ ]:
conn_params = {'host':'localhost',
               'port':3306,
               'user':'root',
               'password':'root',
               'database':"musicdb",
               'autocommit':True}
conn = mariadb.connect(**conn_params)
cur = conn.cursor()

# 사용자 에이전트 설정
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_2)"

options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={user_agent}")

# 크롬실행
driver = webdriver.Chrome("./chromedriver", options=options)

# 150위 까지
for n in tqdm(range(1,152,50)):
    url=f'https://www.melon.com/genre/song_list.htm?gnrCode=GN0900#params%5BgnrCode%5D=GN0900&params%5BdtlGnrCode%5D=GN0901&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex={n}'
    driver.get(url)
    time.sleep(2)
    
    # bs4
    page_source1 = driver.page_source
    html1 = BeautifulSoup(page_source1, 'html.parser')

    link_list=html1.find('tbody').find_all('a',  class_ = 'btn button_icons type03 song_info')

    for i in tqdm(link_list):

        try:

            dic = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_2)'}
            #세부주소
            page=(i['href'].split("'")[-2])
            link=requests.get(f'https://www.melon.com/song/detail.htm?songId={page}',headers=dic)
            time.sleep(1+random.random())
            html=BeautifulSoup(link.text,"html.parser")

            # 가수
            singer=html.find('div',class_='artist').find('span').text

            # 노래제목
            song=" ".join(html.find('div',class_='song_name').text.split()).replace('곡명 ',"")

            # 앨범사진
            album=html.find('div',class_='thumb').find('img')['src']

            # 댓글
            # 1~10페이지
            review1=[]
            for p in tqdm(range(1,11)):
                #세부주소
                page=(i['href'].split("'")[-2])
                link2=(f'https://www.melon.com/song/detail.htm?songId={page}#cmtpgn=&pageNo={p}&sortType=1&srchType=2&srchWord=')
         

                review_= get_review(link2)
                review1.extend(review_)

            print('review1 => ',len(review1))
            # 댓글수 50개 이하 패스
            if len(review1)<=50: raise Exception("댓글수 50개 이하.")

            # 11~20페이지
            review2=[]
            for p in tqdm(range(11,21)):
                #세부주소
                page=(i['href'].split("'")[-2])
                link2=(f'https://www.melon.com/song/detail.htm?songId={page}#cmtpgn=&pageNo={p}&sortType=1&srchType=2&srchWord=')
                
                review_= get_review(link2)
                review2.extend(review_)
            print('review2 => ',len(review2))

            # 댓글수 180개 이하 저장 안함
            if len(review2)<=80: raise Exception("댓글수 80개 이하.")

            sql = "INSERT INTO musictb (singer, song, album, review1, review2) VALUES (?, ?, ?, ?, ?)"

            # 데이터를 튜플로 묶습니다.
            data = (singer, song, album, '|'.join(review1), '|'.join(review2))

            # 쿼리 실행
            cur.execute(sql, data)
            print('db 저장완',singer,song,album,'|'.join(review1), '|'.join(review2))

        except Exception as e:
            print(e)
            print('DB 저장 안함',song)
            pass

cur.close()
conn.close()
